In [6]:
from biopylib.sequence import SQ

def reset_matrix(nrows, ncols):
    res = [ ] 
    for i in range(0, nrows):
        res.append([0]*ncols)
    return res

def print_matrix(mat):
    for i in range(0, len(mat)): print(mat[i])

class MyMotifs:

    def __init__(self, seqs = [], pwm = [], abc = None):
        if seqs:
            self.size = len(seqs[0])
            self.seqs = seqs # objet from class MySeq
            self.abc = seqs[0].count.abc()
            self.get_counts()
            self.get_pwm()
        else:
            self.pwm = pwm
            self.size = len(pwm[0])
            self.abc = abc   
        
    def __len__ (self):
        return self.size        
        
    def get_counts(self):
        self.counts = reset_matrix(len(self.abc), self.size)
        for s in self.seqs:
            for i in range(self.size):
                lin = self.abc.index(s[i])
                self.counts[lin][i] += 1

    # Position Weighted Matrix
                
    def get_pwm(self):
        if(self.counts == None):
            self.get_counts()
        self.pwm = reset_matrix(len(self.abc), self.size)
        for i in range(len(self.abc)):
            for j in range(self.size):
                self.pwm[i][j] = float(self.counts[i][j])/len(self.seqs)
                
    # Return the sequence motif obtained w/ most freq symbol at each
    # position of the motif
                
    def consensus(self):
        res = ""
        for j in range(self.size):
            maxcol = self.counts[0][j]
            maxcoli = 0
            for i in range(1, len(self.abc) ):
                if self.counts[i][j] > maxcol: 
                    maxcol = self.counts[i][j]
                    maxcoli = i
            res += self.abc[maxcoli]        
        return res
    
    # Return the sequence motif obtained w/ the symbol that occurs in at
    # least 50% of the input sequences

    def masked_consensus(self):
        """ returns the sequence motif obtained with the symbol that occurrs in at least 50% of the input sequences"""
        res = ""
        for j in range(self.size):
            maxcol = self.counts[0][j]
            maxcoli = 0
            for i in range(1, len(self.abc) ):
                if self.counts[i][j] > maxcol: 
                    maxcol = self.counts[i][j]
                    maxcoli = i
            if maxcol > len(self.seqs) / 2:
                res += self.abc[maxcoli]        
            else:
                res += "-"
        return res

    def probability_sequence(self, seq):
        res = 1.0
        for i in range(self.size):
            lin = self.abc.index(seq[i])
            res *= self.pwm[lin][i]
        return res
    
    def probability_all_positions(self, seq):
        res = []
        for k in range(len(seq)-self.size+1):
            res.append(self.probability_sequence(seq))
        return res

    def most_probable_sequence(self, seq):
        """ Returns the index of the most probable sub-sequence of the input sequence"""
        maximum = -1.0
        maxind = -1
        for k in range(len(seq)-self.size):
            p = self.probability_sequence(seq[k:k+ self.size])
            if(p > maximum):
                maximum = p
                maxind = k
        return maxind

    def create_motif(self, seqs):
        l = []
        for s in seqs: 
            ind = self.most_probable_sequence(s.seq)
            subseq = SQ(seq=s[ind:(ind+self.size)],seq_type='dna' )
            l.append(subseq)
        
        return MyMotifs(l) 


def test():
    seq1 = SQ("AAAGTT")
    seq2 = SQ("CACGTG")
    seq3 = SQ("TTGGGT")
    seq4 = SQ("GACCGT")
    seq5 = SQ("AACCAT")
    seq6 = SQ("AACCCT")
    seq7 = SQ("AAACCT")
    seq8 = SQ("GAACCT")
    lseqs = [seq1, seq2, seq3, seq4, seq5, seq6, seq7, seq8]
    motifs = MyMotifs(lseqs)
    print ("Counts matrix")
    print_matrix (motifs.counts)
    print ("PWM")
    print_matrix (motifs.pwm)
    
    print(motifs.probability_sequence("AAACCT"))
    print(motifs.probability_sequence("ATACAG"))
    print(motifs.most_probable_sequence("CTATAAACCTTACATC"))
    
    # for s in lseqs:
    #     print (s)
    # print ("Consensus sequence")
    # print(motifs.consensus())
    # print ("Masked Consensus sequence")
    # print(motifs.masked_consensus())

    # s1 = SQ("TAAAGTTATGA")
    # s2 = SQ("ATGACACGTG")
    # s3 = SQ("TTTGGGTAT")

    # newmotif = motifs.create_motif([s1,s2,s3])
    # print_matrix(newmotif.counts)
    # print(newmotif.most_probable_sequence("AAAACT"))
    
test()

Counts matrix
[4, 7, 3, 0, 1, 0]
[1, 0, 4, 5, 3, 0]
[2, 0, 1, 3, 2, 1]
[1, 1, 0, 0, 2, 7]
PWM
[0.5, 0.875, 0.375, 0.0, 0.125, 0.0]
[0.125, 0.0, 0.5, 0.625, 0.375, 0.0]
[0.25, 0.0, 0.125, 0.375, 0.25, 0.125]
[0.125, 0.125, 0.0, 0.0, 0.25, 0.875]
0.0336456298828125
0.0002288818359375
4
